**State Bank Deposits by Bank**

*Loading in Data*

In [1]:
# Import
import pandas as pd

# Reading in large dataframe
# See data cleaning
name_df = pd.read_csv("../data/quarterly_banks_by_state.csv", index_col=False)

# Labeling the top banks
top_banks = [
    "JPMORGAN CHASE BANK NA",
    "BANK OF AMERICA NA",
    "WELLS FARGO BANK NA",
    "CITIBANK NATIONAL ASSN",
]
name_df["top_bank"] = [
    bank if bank in top_banks else "Other" for bank in name_df["NAME"]
]

# Grouped Dataframe
df2 = name_df.groupby(["STNAME", "top_bank", "date", "decade"])[
    "DEPDOM", "ASSET"
].apply(lambda x: x.astype(int).sum())
df2 = df2.reset_index()

# Adding our theme colors for each bank
color_col = []

for bank in df2["top_bank"]:
    if bank == "JPMORGAN CHASE BANK NA":
        color = "#117ACA"
    elif bank == "BANK OF AMERICA NA":
        color = "#E61030"
    elif bank == "WELLS FARGO BANK NA":
        color = "#ffcc02"
    elif bank == "CITIBANK NATIONAL ASSN":
        color = "#003A72"
    else:
        color = "#118C4F"

    color_col.append(color)

df2["color"] = color_col
df2 = df2.reset_index()

# Displaying
df2.head()


/var/folders/hg/dd3yfd8j7vx8qtmvm42400j80000gn/T/ipykernel_18467/2714565487.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2 = name_df.groupby(["STNAME", "top_bank", "date", "decade"])["DEPDOM", "ASSET"].apply(


,index,STNAME,top_bank,date,decade,DEPDOM,ASSET,color
0,0,ALABAMA,Other,1992-03-31,1990,28888570,34804559,#118C4F
1,1,ALABAMA,Other,1992-06-30,1990,28782222,35151498,#118C4F
2,2,ALABAMA,Other,1992-09-30,1990,29034487,35810540,#118C4F
3,3,ALABAMA,Other,1992-12-31,1990,30059749,36943922,#118C4F
4,4,ALABAMA,Other,1993-03-31,1990,29888130,37040762,#118C4F


*Plot*

In [16]:
# Imports
import plotly.graph_objects as go
import plotly.express as px

# initialize figure
fig = go.Figure()


# Empty Lists
sliders = []
buttonList = []

# Filtering to just the states we want to focus on
states = ["SOUTH DAKOTA", "NEW YORK", "CALIFORNIA", "TEXAS", "NORTH CAROLINA", "OHIO"]

df2 = df2[[True if state in states else False for state in df2["STNAME"]]]

# Creating an empty visibility list of False values that we will fill
# numBank = 5
numStates = len(df2.STNAME.unique())
visibility_list = [False] * numStates

# Counter for instances
counter = 0

# Add traces, one for each slider step
for i, state in enumerate(df2.STNAME.unique()):
    steps = []

    # Filtered state dataframe
    stateDF = df2[df2.STNAME == state]

    # Displaying the first year for each state
    state_first_visibility = visibility_list.copy()
    state_first_visibility[counter] = True
    stateDF.sort_values(by="date", ascending=True, inplace=True)

    # Each  has its own visibility list
    visibility_list1 = visibility_list.copy()
    visibility_list1[counter] = True
    # Adding to the counter
    counter += 1

    # # Steps on the slider
    # step = {
    #     "method": "update",
    #     "args": [{"visible": visibility_list1}],
    #     "label": str(decade),
    # }
    # steps.append(step)

    # ID
    id = str(state)

    # Making the bank names prettier
    name_col = []

    for bank in stateDF["top_bank"]:
        if bank == "JPMORGAN CHASE BANK NA":
            name = "JP Morgan Chase"
        elif bank == "BANK OF AMERICA NA":
            name = "Bank of America"
        elif bank == "WELLS FARGO BANK NA":
            name = "Wells Fargo"
        elif bank == "CITIBANK NATIONAL ASSN":
            name = "Citibank"
        else:
            name = "All Other Banks"

        name_col.append(name)

    stateDF["pretty_name"] = name_col
    stateDF = stateDF.reset_index()

    # Adding the trace
    fig.add_trace(
        go.Scatter(
            uid=id,
            visible=False,
            x=stateDF["date"],
            y=stateDF["DEPDOM"],
            mode="markers",
            marker_color=stateDF["color"],
            showlegend=True,
            name=stateDF["pretty_name"][i],
        )
    )

    # Slider functionality removed to see full picture
    # # Creating a slider for the decade
    # slider1 = [
    #     dict(
    #         active=0,
    #         currentvalue={"prefix": "Decade: ", "suffix": "s"},
    #         pad={"t": 50},
    #         steps=steps,
    #     )
    # ]
    # sliders.append(slider1)

    # Making the button
    tmpbutton = dict(
        label=str(state),
        method="update",
        args=[{"visible": state_first_visibility}],
        # Slider functionality
        # args=[{"visible": state_first_visibility}, {"sliders": slider1}],
    )

    # Adding the button
    buttonList.append(tmpbutton)

# print("OUT OF LOOP")
# Make First trace visible
fig.data[0].visible = True

# DEFINE ONE SLIDE FOR EACH BUTTON
# fig.update_layout(sliders=sliders[0])

# print("LAYOUT UPDATED")
# Adding all of the buttons
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttonList,
            direction="down",
            showactive=True,
            pad={"r": 10, "t": 10},
            x=0.1,
            xanchor="left",
            y=1.2,
            yanchor="top",
        ),
    ],
    xaxis_title="Date",
    yaxis_title="Total Deposits (USD)",
    title="Total Deposits by State",
    template="plotly_white",
)


# print("ABOUT TO DISPLAY")
# Updating labels
fig.update_traces(
    hovertemplate="Date: %{x} <br>Total Deposits: %{y:$.0f}<extra></extra>"
)


# Displaying
fig.show()

/var/folders/hg/dd3yfd8j7vx8qtmvm42400j80000gn/T/ipykernel_18467/2698349587.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/hg/dd3yfd8j7vx8qtmvm42400j80000gn/T/ipykernel_18467/2698349587.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/hg/dd3yfd8j7vx8qtmvm42400j80000gn/T/ipykernel_18467/2698349587.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c